In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

import scipy.ndimage as ndi

from ImageD11.nbGui import nb_utils as utils
import ImageD11.sinograms.dataset

from ImageD11.grain import grain
from ImageD11 import unitcell
%matplotlib ipympl

In [ ]:
# USER: Pass path to dataset file

dset_file = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

ds = ImageD11.sinograms.dataset.load(dset_file)

sample = ds.sample
dataset = ds.dsname
processed_data_root_dir = ds.analysisroot

print(ds)
print(ds.shape)

In [ ]:
# define your reference unit cell for RGB colour plotting

cf_2d = ds.get_cf_2d_from_disk()
cf_2d.parameters.loadparameters(ds.parfile)

cf_pars = cf_2d.parameters.get_parameters()
spacegroup = 227  # spacegroup for Si
cf_pars["cell_lattice_[P,A,B,C,I,F,R]"] = spacegroup

ref_ucell = ImageD11.unitcell.unitcell_from_parameters(cf_pars)

In [ ]:
# pbp_array = np.loadtxt('/home/esrf/james1997a/Code/ImageD11/ImageD11/nbGui/S3DXRD/grainsout_all.pbp')

pbp_array = np.loadtxt(ds.pbpfile)
ubi = {}
i,j,n,u,ubi[0,0],ubi[0,1],ubi[0,2],ubi[1,0],ubi[1,1],ubi[1,2],ubi[2,0],ubi[2,1],ubi[2,2] = pbp_array.T
i = i.astype(int) 
i -= i.min()   # position in array / space
j = j.astype(int) 
j -= j.min()
n = n.astype(int)         # total peaks indexing with hkl==int with 0.03
u = u.astype(int)         # unique (h,k,l) labels on indexed peaks

In [ ]:
NY = int(i.max()-i.min())+1
i.min(),i.max(),NY

In [ ]:
fig, ax = plt.subplots()
ax.hist(u, bins=128)
ax.set_xlabel('Unique spots per pixel')
ax.set_ylabel('Count')
plt.show()

In [ ]:
minpeaks = 20

uni = np.ones((NY,NY))
uni.fill(minpeaks) # peak cutorr
npk = np.zeros((NY,NY))
ubifit = np.zeros((NY,NY,3,3), float)
ubifit.fill(np.nan)
ubilist = []
for k in range(len(i)):
    if u[k] > uni[i[k],j[k]]:
        uni[i[k],j[k]] = u[k]
        npk[i[k],j[k]] = n[k]
        for ii in range(3):
            for jj in range(3):
                ubifit[i[k],j[k],ii,jj] = ubi[ii,jj][k]
        ubilist.append(ubifit[i[k],j[k]].copy())

In [ ]:
def plot_inverse_pole_figure(UBs, ref_ucell, axis=np.array([0., 0, 1])):
    # get a meta orientation for all the grains
    meta_orien = ref_ucell.get_orix_orien(UBs)

    try:
        from orix.vector.vector3d import Vector3d
    except ImportError:
        raise ImportError("Missing diffpy and/or orix, can't compute orix phase!")

    ipf_direction = Vector3d(axis)

    # get the RGB colours
    rgb = ref_ucell.get_ipf_colour_from_orix_orien(meta_orien, axis=ipf_direction)

    # scatter the meta orientation using the colours
    meta_orien.scatter("ipf", c=rgb, direction=ipf_direction, s=1)

    
def plot_all_ipfs(UBs, ref_ucell):
    plot_inverse_pole_figure(UBs, ref_ucell, axis=np.array([1., 0, 0]))
    plot_inverse_pole_figure(UBs, ref_ucell, axis=np.array([0., 1, 0]))
    plot_inverse_pole_figure(UBs, ref_ucell, axis=np.array([0., 0, 1]))

In [ ]:
ublist = [np.linalg.inv(ubi) for ubi in ubilist]
ubarray = np.array(ublist)

plot_all_ipfs(ubarray, ref_ucell)

In [ ]:
def normalise( r ):
    negatives = r [r < 0 ]
    noise = np.std(negatives)
    s = ndi.uniform_filter( r, 5 )
    positives =  s > noise
    height = r[positives].mean()
    return np.where(s>noise, r/r.max(), 0)

In [ ]:
r = npk.copy()
ubisel = ubifit.copy()

In [ ]:
# container to store rgb values, shape (axis, i, j, RGB)
rgb = np.zeros( (3, ubisel.shape[0], ubisel.shape[1], 3 ))

# iterate over the axes

for k,axis in enumerate( ((1,0,0),(0,1,0),(0,0,1))):
    UB_ij = []

    for i in range(ubisel.shape[0]):
        for j in range(ubisel.shape[1]):

                if np.isnan( ubisel[i,j,0,0] ):
                    rgb[k,i,j] = np.nan
                else:
                    this_ubi = ubisel[i,j]
                    this_ub = np.linalg.inv(this_ubi)

                    UB_ij.append((this_ub, i, j))
    
    # get_ipf_colour is slow but vectorised, so we can call it once for all UB matrices with this IPF axis
    UBs_flat = np.array([UB for (UB, i, j) in UB_ij])
    cols_flat = ref_ucell.get_ipf_colour(UBs_flat, axis)
    
    # populate rgb array
    for (UB, i, j), col in zip(UB_ij, cols_flat):
        rgb[k, i, j] = col

In [ ]:
fig, a = plt.subplots(2,2,sharex=True,sharey=True, figsize=(10,10), constrained_layout=True)
a[0][0].imshow(rgb[0], origin="lower")
a[0][0].set_title("IPF X")
a[0][1].imshow(rgb[1], origin="lower")
a[0][1].set_title("IPF Y")
a[1][0].imshow(rgb[2], origin="lower")
a[1][0].set_title("IPF Z")
m = r > 0.02
a[1][1].imshow(np.where(m, r, np.nan), origin="lower")
a[1][1].set_title("Number of peaks")
# for ax in a.ravel():
#     ax.set(yticks=[], xticks=[])
#     ax.invert_yaxis()

plt.show()